In [1]:
import json
import pathlib
from tqdm import tqdm

In [2]:
# Construct single json file
data_dir = pathlib.Path('./data/')
json_paths = sorted(data_dir.glob('*.json'))
out_json = pathlib.Path('./merged.json')
data = []
for path in tqdm(json_paths):
    with path.open() as f:
        data.append(json.load(f))
with out_json.open('w') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

100%|██████████| 1266/1266 [00:00<00:00, 11875.13it/s]


In [2]:
%%bash
# Start solr
../solr-7.3.0/bin/solr start
# Create core
../solr-7.3.0/bin/solr create -c recipe

*** [WARN] *** Your open file limit is currently 1024.  
 It should be set to 65000 to avoid operational disruption. 
 If you no longer wish to see this warning, set SOLR_ULIMIT_CHECKS to false in your profile or solr.in.sh
*** [WARN] ***  Your Max Processes Limit is currently 31295. 
 It should be set to 65000 to avoid operational disruption. 
 If you no longer wish to see this warning, set SOLR_ULIMIT_CHECKS to false in your profile or solr.in.sh
Waiting up to 180 seconds to see Solr running on port 8983 [|]   [/]   [-]   [\]   [|]   [/]   [-]   [\]   [|]   [/]   [-]   [\]   [|]   [/]   [-]   [\]  
Started Solr server on port 8983 (pid=891). Happy searching!

    WARNING: Using _default configset with data driven schema functionality. NOT RECOMMENDED for production use.
         To turn off: bin/solr config -c recipe -p 8983 -property update.autoCreateFields -value false

Created new c

In [3]:
%%bash
# Commit
curl 'http://localhost:8983/solr/recipe/update?commit=true' \
    --data-binary @./merged.json \
    -H 'Content-type:application/json'

{
  "responseHeader":{
    "status":0,
    "QTime":2691}}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2708k  100    58  100 2708k     21  1003k  0:00:02  0:00:02 --:--:-- 1003k


In [42]:
%%bash

curl http://localhost:8983/solr/recipe/query -d '
{
  "query" : "ingr:pasta AND n_ingr:[74] TO *]",
  "fields" : ["n_ingr"]
}'

{
  "responseHeader":{
    "status":400,
    "QTime":2,
    "params":{
      "json":"\n{\n  \"query\" : \"ingr:pasta AND n_ingr:[[4] TO [15]]\",\n  \"fields\" : [\"n_ingr\"]\n}"}},
  "error":{
    "metadata":[
      "error-class","org.apache.solr.common.SolrException",
      "root-error-class","org.apache.solr.parser.ParseException"],
    "msg":"org.apache.solr.search.SyntaxError: Cannot parse 'ingr:pasta AND n_ingr:[[4] TO [15]]': Encountered \" \"]\" \"] \"\" at line 1, column 25.\nWas expecting:\n    \"TO\" ...\n    ",
    "code":400}}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   624  100   545  100    79  45416   6583 --:--:-- --:--:-- --:--:-- 52000


In [70]:
%%bash
# Delete core
curl "http://localhost:8983/solr/admin/cores?action=UNLOAD&core=recipe&deleteIndex=true&deleteDataDir=true&deleteInstanceDir=true"
# Shutdown
../solr-7.3.0/bin/solr stop -all

{
  "responseHeader":{
    "status":0,
    "QTime":20}}
Sending stop command to Solr running on port 8983 ... waiting up to 180 seconds to allow Jetty process 13340 to stop gracefully.
    

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    56  100    56    0     0   1806      0 --:--:-- --:--:-- --:--:--  1806
